In [4]:
import pandas as pd
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report



In [5]:
userData = pd.read_csv('../data/userData.csv')
userData = userData.drop(columns=['weekday'])
userData

,food_name,time_of_day
0,chipotle chicken and queso bowl,Morning
1,kodiak cake waffles,Night
2,chipotle bowl chicken and queso,Evening
3,starbucks caramel macchiato,Morning
4,chicken tacos,Evening
...,...,...
374,mcdonald’s cheeseburger,Evening
375,small flour tortillas,Evening
376,chicken,Evening
377,2 percent fairlife chocolate milk,Evening


In [6]:

# Assuming you have your data loaded into a DataFrame called 'userData'
# Let's encode categorical variables using one-hot encoding
df_encoded = pd.get_dummies(userData, columns=['time_of_day'])

# Separate features and target variable
X = df_encoded.drop('food_name', axis=1)
y = df_encoded['food_name']


times_of_day = ['Morning', 'Afternoon', 'Evening', 'Night']

# Generate all possible combinations of day and time
all_permutations = list(times_of_day)

# print(all_permutations)

# Create a DataFrame to store the permutations
manual_test = pd.DataFrame(all_permutations, columns=['time_of_day'])
manual_encoded = pd.get_dummies(manual_test, columns=['time_of_day'])



# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=43)

# Initialize the Multi-Layer Perceptron (MLP) classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=43)

# Train the classifier
mlp_classifier.fit(X_train, y_train)

# Make predictions on the test set with probability estimates
y_pred_proba = mlp_classifier.predict_proba(manual_encoded)

# Specify how many top predictions you want to consider
top_n = 5
# Get the indices of the top N predictions for each sample
top_n_indices = (-y_pred_proba).argsort(axis=1)[:, :top_n]

# Get the top N predictions for each sample
top_n_predictions = mlp_classifier.classes_[top_n_indices]

# Assuming you have stored the day and time_of_day columns separately
# day_cols = ['weekday_Monday', 'weekday_Tuesday', 'weekday_Wednesday', 'weekday_Thursday', 'weekday_Friday', 'weekday_Saturday', 'weekday_Sunday']
time_cols = ['time_of_day_Morning', 'time_of_day_Afternoon', 'time_of_day_Evening', 'time_of_day_Night']

# Print the top N predictions for each sample along with their parameters and probabilities

result = []

with open("../out/time_based_output_mlp.txt", "w") as f:
    for i in range(len(manual_encoded)):
        # day = [col.split('_')[1] for col in day_cols if manual_encoded.iloc[i][col] == 1][0]
        time_of_day = [col.split('_')[3] for col in time_cols if manual_encoded.iloc[i][col] == 1][0]

        # f.write("Day: {}\n".format(day))
        f.write("Time of Day: {}\n".format(time_of_day))
        f.write("Predictions:\n")
        for j in range(top_n):
            f.write("    - Prediction {}: {}, Probability: {:.2f}%\n".format(j + 1, top_n_predictions[i][j], y_pred_proba[i][top_n_indices[i][j]] * 100))
        f.write("\n")
        result.append({
            # 'day': day,
            'time_of_day': time_of_day,
        })
        for j in range(top_n):  # Assuming you want top 5 predictions
            result[-1][f'prediction{j+1}'] = top_n_predictions[i][j]
            result[-1][f'probability{j+1}'] = y_pred_proba[i][top_n_indices[i][j]] * 100

df_result = pd.DataFrame(result)
df_result

# df_result.to_csv('time_based_results.csv', index=False)




,time_of_day,prediction1,probability1,prediction2,probability2,prediction3,probability3,prediction4,probability4,prediction5,probability5
0,Morning,cups of milk,11.327921,chipotle chicken and queso bowl,10.993025,protein shake with milk and peanut butter,10.877607,chicken,5.685995,milk,5.602036
1,Afternoon,chick-fil-a sandwich,8.706082,sweet tea,8.438889,kodiak cake waffles,3.663862,borden chocolate milk,3.510497,milk,2.697586
2,Evening,chick-fil-a sandwich,23.875429,sweet tea,21.620792,chick-fil-a granola parfait,2.208156,banana,2.182737,chipotle chicken and queso bowl,1.781237
3,Night,kodiak cake waffles,13.795003,4oz chicken,7.440876,cup of milk,7.296546,fettuccine,7.218894,bowl of oatmeal topped with fresh berries and ...,7.069864
